In [23]:
using Test

include("./computer.jl")

input_data = open("input07.txt") do file
    data = readline(file)
    entries = split(data, ",")
    numbers = map(x->parse(Int64, x), entries)
    numbers
end


input_data;

In [24]:
function run_phasers_async(input_data, phaser_config::Array{Int64, 1})
    tasks = []
    num_phasers = length(phaser_config)

    local input_value::Int64 = 0
    local output_value::Int64 = 0
    com_channels = [Channel{Int64}(Inf) for i = 1:num_phasers]
    for (i::Int64, phase::Int64) in enumerate(phaser_config)
        put!(com_channels[i], phase)

    end
    put!(com_channels[1], 0)
    tasks = Task[]
    
    @sync for (i::Int64, phase::Int64) in enumerate(phaser_config)
        # @info("$i, $input_value, $output_value")
        #@info("$input_data")
        out_channel_index = (i < num_phasers) ? i+1 : 1
        task = @async run_program_async(input_data, com_channels[i], com_channels[out_channel_index])
        push!(tasks, task)
        #@info("$out.output")
    end
    
    output_value = take!(com_channels[1])
    return output_value
end

function maxout_phasers_async(input_data, phaser_possibilities=5:9)
    # generate 5-digit input
    num_phasers = length(phaser_possibilities)

    iterator = Iterators.product(ntuple(i->phaser_possibilities, num_phasers)...)
    current_max = 0
    current_conf = undef

    
    for it in iterator
        if length(Set(it)) < num_phasers
            continue
        end
        output_value = run_phasers_async(input_data, collect(it))
        
        if output_value > current_max
            current_max = output_value
            current_conf = it
        end
    end
    return current_max, current_conf
end

maxout_phasers_async (generic function with 2 methods)

In [25]:
test_data = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]


@test run_phasers_async(test_data, [9,8,7,6,5]) == 139629729
maxout_phasers_async(input_data)

(1336480, (9, 6, 5, 7, 8))

In [2]:
channel1 = Channel{Int64}(Inf)
channel2 = Channel{Int64}(Inf)
channels = [channel1, channel2]
put!(channel1, 5)

function test_channel(channel_in, channel_out)
    println(current_task())
    println("taking value")
    value = take!(channel_in)
    println("taken value: $value")
    put!(channel_out, 42)
    value = take!(channel_in)
    println("taken value: $value")

    put!(channel_out, 37)


end

@sync begin
    @async test_channel(channels[1], channels[2])
    @async test_channel(channels[2], channels[1])

end


Task (runnable) @0x00007f5718372230
taking value
taken value: 5
Task (runnable) @0x00007f57183724a0
taking value
taken value: 42
taken value: 42
taken value: 37


Task (done) @0x00007f57183724a0